In [1]:
from dotenv import load_dotenv
import os
from groq import Groq


load_dotenv()

True

In [2]:
api_key = os.environ.get('GROQ_API_KEY')
client = Groq(api_key=api_key)
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Requirements**: Many applications, such as voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications such as language translation, sentiment analysis, and text summarization.
4. **Energy Efficiency**: Fast

In [3]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_city_population:
e.g. get_city_population: Tokyo
returns the population of the city

Example session:

Question: What is the population of New York City plus 1 million?
Thought: I need to find the population of New York City
Action: get_city_population: New York City
PAUSE 

You will be called again with this:

Observation: 8,804,190

Thought: I need to add 1 million to this number
Action: calculate: 8804190 + 1000000
PAUSE

You will be called again with this: 

Observation: 9,804,190

If you have the answer, output it as the Answer.

Answer: The population of New York City plus 1 million is 9,804,190.

Now it's your turn:
"""

def calculate(operation: str) -> float:
    return {
            "data": [
                {
                    "data": eval(operation),
                    "response_type": "ResponseTypes.CODE.value",
                }
            ],
            "tool_called": "ToolCalled.Trino_Data_Virtualization.value"
        }
    

def get_city_population(city: str) -> int:
    match city.lower():
        case "tokyo":
            return 37_400_068
        case "delhi":
            return 31_399_000
        case "shanghai":
            return 27_058_000
        case "são paulo":
            return 22_043_000
        case "mexico city":
            return 21_782_000
        case "cairo":
            return 20_901_000
        case "mumbai":
            return 20_411_000
        case "beijing":
            return 20_384_000
        case "dhaka":
            return 20_283_000
        case "osaka":
            return 19_165_000
        case _:
            return 0

In [6]:
neil_tyson = Agent(client=client, system=system_prompt)

In [8]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_city_population"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


loop(query="What is the population of tokyo City plus 1 million?")

Thought: I need to find the population of Tokyo City
Action: get_city_population: Tokyo
PAUSE
Observation: 37400068
Thought: I need to add 1 million to this number
Action: calculate: 37400068 + 1000000
PAUSE
Observation: {'data': [{'data': 38400068, 'response_type': 'ResponseTypes.CODE.value'}], 'tool_called': 'ToolCalled.Trino_Data_Virtualization.value'}
Thought: I have the result of the calculation, which is the population of Tokyo City plus 1 million 

Answer: The population of Tokyo City plus 1 million is 38400068.
